### Test if Parser Works

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://www.indexmundi.com/facts/united-states/quick-facts/alabama/median-household-income#map"
response = requests.get(url)

# if successful parse the download into a BeautifulSoup object 
if response.status_code == 200:
    soup = BeautifulSoup(response.content, "html.parser")
    
# find the object with HTML id tableTab
table = soup.find('div',{'id':'tableTab'})

# loop through all the rows and pull the text
AL = []
for row in table.find_all('tr')[1:]:
    column_marker = 0
    columns = row.find_all('td')
    AL.append([column.get_text() for column in columns])
    
# df = pd.DataFrame(new_table, columns=['County','Income'])
# df['Alabama'] = df['Alabama'].str.replace('\n','')
# df

In [2]:
type(AL[0])

list

### Create URLs for each State

In [3]:
states = ["Alabama","Alaska","Arizona","Arkansas","California","Colorado",
  "Connecticut","Delaware","District-of-Columbia","Florida","Georgia","Hawaii","Idaho","Illinois",
  "Indiana","Iowa","Kansas","Kentucky","Louisiana","Maine","Maryland",
  "Massachusetts","Michigan","Minnesota","Mississippi","Missouri","Montana",
  "Nebraska","Nevada","New-Hampshire","New-Jersey","New-Mexico","New-York",
  "North-Carolina","North-Dakota","Ohio","Oklahoma","Oregon","Pennsylvania",
  "Rhode-Island","South-Carolina","South-Dakota","Tennessee","Texas","Utah",
  "Vermont","Virginia","Washington","West-Virginia","Wisconsin","Wyoming"]

In [4]:
len(states)

51

In [5]:
urls = []
for s in states:
    url = "https://www.indexmundi.com/facts/united-states/quick-facts/"+s+"/median-household-income#map"
    urls.append(url)

### Create a Dictionary with County Data for Each State

In [9]:
from tqdm import tqdm_notebook as tqdm

In [10]:
all_counties = []
for url in tqdm(urls):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        
    table = soup.find('div',{'id':'tableTab'})
    for row in table.find_all('tr'):
        column_marker = 0            
        columns = row.find_all('td')
        all_counties.append([column.get_text() for column in columns])

In [11]:
len(all_counties)

3173

In [18]:
# remove duplicates 

biglist = [[]]
for i in all_counties:
    if not i:
        biglist.append([])
    else:
        biglist[-1].append(i)
print(biglist)

[[], [['Autauga', '58,786'], ['Baldwin', '55,962'], ['Barbour', '34,186'], ['Bibb', '45,340'], ['Blount', '48,695'], ['Bullock', '32,152'], ['Butler', '39,109'], ['Calhoun', '45,197'], ['Chambers', '39,872'], ['Cherokee', '41,014'], ['Chilton', '45,182'], ['Choctaw', '34,884'], ['Clarke', '36,127'], ['Clay', '41,225'], ['Cleburne', '40,188'], ['Coffee', '53,155'], ['Colbert', '47,558'], ['Conecuh', '33,177'], ['Coosa', '38,301'], ['Covington', '40,601'], ['Crenshaw', '39,812'], ['Cullman', '42,558'], ['Dale', '45,960'], ['Dallas', '31,602'], ['DeKalb', '39,233'], ['Elmore', '59,347'], ['Escambia', '35,000'], ['Etowah', '44,023'], ['Fayette', '37,458'], ['Franklin', '40,459'], ['Geneva', '38,142'], ['Greene', '21,804'], ['Hale', '30,571'], ['Henry', '48,610'], ['Houston', '45,496'], ['Jackson', '39,816'], ['Jefferson', '51,979'], ['Lamar', '36,756'], ['Lauderdale', '46,265'], ['Lawrence', '44,515'], ['Lee', '48,056'], ['Limestone', '56,460'], ['Lowndes', '30,833'], ['Macon', '32,074'], 

In [19]:
# remove first empty bracket
counties_list = biglist[1:]

In [20]:
len(counties_list)

51

In [23]:
flat_list = []
for sublist in counties_list:
    for item in sublist:
        flat_list.append(item)

In [24]:
state_incomes = dict(zip(states,counties_list))

In [25]:
state_incomes

{'Alabama': [['Autauga', '53,682'],
  ['Baldwin', '50,221'],
  ['Barbour', '32,911'],
  ['Bibb', '36,447'],
  ['Blount', '44,145'],
  ['Bullock', '32,033'],
  ['Butler', '29,918'],
  ['Calhoun', '39,962'],
  ['Chambers', '32,402'],
  ['Cherokee', '34,907'],
  ['Chilton', '41,250'],
  ['Choctaw', '33,941'],
  ['Clarke', '29,357'],
  ['Clay', '34,002'],
  ['Cleburne', '38,019'],
  ['Coffee', '43,768'],
  ['Colbert', '39,077'],
  ['Conecuh', '24,658'],
  ['Coosa', '37,277'],
  ['Covington', '35,869'],
  ['Crenshaw', '36,275'],
  ['Cullman', '38,847'],
  ['Dale', '44,889'],
  ['Dallas', '26,519'],
  ['DeKalb', '37,851'],
  ['Elmore', '54,766'],
  ['Escambia', '30,687'],
  ['Etowah', '38,013'],
  ['Fayette', '33,838'],
  ['Franklin', '36,415'],
  ['Geneva', '35,625'],
  ['Greene', '24,226'],
  ['Hale', '30,051'],
  ['Henry', '41,650'],
  ['Houston', '40,948'],
  ['Jackson', '37,634'],
  ['Jefferson', '45,429'],
  ['Lamar', '36,028'],
  ['Lauderdale', '42,844'],
  ['Lawrence', '38,551'],
  [

In [26]:
everycounty_income = []
for state, counties in state_incomes.items():
    for county in counties:
        county_info = []
        county_info.append(state)
        county_info.append(county[0])
        county_info.append(county[1])
        everycounty_income.append(county_info)

In [27]:
incomebycounty = pd.DataFrame(everycounty_income, columns = ['State', 'County', 'Avg_Income'])

In [28]:
incomebycounty.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3143 entries, 0 to 3142
Data columns (total 3 columns):
State         3143 non-null object
County        3143 non-null object
Avg_Income    3143 non-null object
dtypes: object(3)
memory usage: 73.7+ KB


In [29]:
incomebycounty['Avg_Income'] = incomebycounty['Avg_Income'].str.replace(',','')
incomebycounty['Avg_Income'] = incomebycounty.Avg_Income.astype(float)

In [30]:
incomebycounty.head()

,State,County,Avg_Income
0,Alabama,Autauga,53682.0
1,Alabama,Baldwin,50221.0
2,Alabama,Barbour,32911.0
3,Alabama,Bibb,36447.0
4,Alabama,Blount,44145.0


In [31]:
regioncounty = incomebycounty

In [32]:
regioncounty.head()

,State,County,Avg_Income
0,Alabama,Autauga,53682.0
1,Alabama,Baldwin,50221.0
2,Alabama,Barbour,32911.0
3,Alabama,Bibb,36447.0
4,Alabama,Blount,44145.0


In [34]:
regioncounty.County = [str(col) + ' County' for col in regioncounty.County]

In [35]:
regioncounty.head()

,State,County,Avg_Income
0,Alabama,Autauga County,53682.0
1,Alabama,Baldwin County,50221.0
2,Alabama,Barbour County,32911.0
3,Alabama,Bibb County,36447.0
4,Alabama,Blount County,44145.0


#### SQL STUFF!!!

In [39]:
import mysql.connector
import config

In [40]:
#set up mysql connection
cnx = mysql.connector.connect(
   host = config.host,
   user = config.user,
   password = config.password)
cursor = cnx.cursor()

In [41]:
#specify database for connection
cnx = mysql.connector.connect(
   host = config.host,
   user = config.user,
   password = config.password,
   database = "water")
cursor = cnx.cursor()

In [158]:
create_query = """
     CREATE TABLE countyincomes2
   ( State VARCHAR(50),
     County VARCHAR(50),
     Avg_Income FLOAT(10),
     region VARCHAR(2)
     )
   """

In [159]:
cursor.execute(create_query)

In [36]:
subset = incomebycounty[['State', 'County', 'Avg_Income']]
tuples = [tuple(x) for x in subset.values]

In [37]:
def insert_datas(dataframe):
    tuples = list(dataframe.itertuples(index=False, name=None))
    insert_statement = """INSERT INTO countyincomes (State,
                                               County,
                                               Avg_Income)
                           VALUES (%s, %s, %s);"""
    cursor.executemany(insert_statement, tuples)
    cnx.commit()

In [42]:
insert_datas(regioncounty)

##### States By Region

In [43]:
statesabv = {
        'AK': 'O','AL': 'S','AR': 'S','AS': 'O','AZ': 'W','CA': 'W','CO': 'W',
        'CT': 'N','DC': 'N','DE': 'N','FL': 'S','GA': 'S','GU': 'O','HI': 'O',
        'IA': 'M','ID': 'W','IL': 'M','IN': 'M','KS': 'M','KY': 'S','LA': 'S',
        'MA': 'N','MD': 'N','ME': 'N','MI': 'W','MN': 'M','MO': 'M','MP': 'O',
        'MS': 'S','MT': 'W','NA': 'O','NC': 'S','ND': 'M','NE': 'W','NH': 'N',
        'NJ': 'N','NM': 'W','NV': 'W','NY': 'N','OH': 'M','OK': 'S','OR': 'W',
        'PA': 'N','PR': 'O','RI': 'N','SC': 'S','SD': 'M','TN': 'S','TX': 'S',
        'UT': 'W','VA': 'S','VI': 'O','VT': 'N','WA': 'W','WI': 'M','WV': 'S',
        'WY': 'W'
        }

In [44]:
statesab2 = {
        'AK': 'Alaska','AL': 'Alabama', 'AR': 'Arkansas', 'AS': 'American Samoa',
        'AZ': 'Arizona','CA': 'California','CO': 'Colorado','CT': 'Connecticut',
        'DC': 'District of Columbia','DE': 'Delaware','FL': 'Florida','GA': 'Georgia',
        'GU': 'Guam','HI': 'Hawaii','IA': 'Iowa','ID': 'Idaho','IL': 'Illinois',
        'IN': 'Indiana','KS': 'Kansas','KY': 'Kentucky','LA': 'Louisiana',
        'MA': 'Massachusetts','MD': 'Maryland','ME': 'Maine','MI': 'Michigan',
        'MN': 'Minnesota','MO': 'Missouri','MP': 'Northern Mariana Islands',
        'MS': 'Mississippi','MT': 'Montana','NA': 'National','NC': 'North Carolina',
        'ND': 'North Dakota','NE': 'Nebraska','NH': 'New Hampshire','NJ': 'New Jersey',
        'NM': 'New Mexico','NV': 'Nevada','NY': 'New York','OH': 'Ohio',
        'OK': 'Oklahoma','OR': 'Oregon','PA': 'Pennsylvania','PR': 'Puerto Rico',
        'RI': 'Rhode Island','SC': 'South Carolina','SD': 'South Dakota',
        'TN': 'Tennessee','TX': 'Texas','UT': 'Utah','VA': 'Virginia',
        'VI': 'Virgin Islands','VT': 'Vermont','WA': 'Washington',
        'WI': 'Wisconsin','WV': 'West Virginia','WY': 'Wyoming'
}

In [45]:
state_regions = [statesab2, statesabv]
d = {}
for k in statesab2.keys():
    d[k] = tuple(d[k] for d in state_regions)

In [46]:
region_list = list(d.values())
region_list

[('Alaska', 'O'),
 ('Alabama', 'S'),
 ('Arkansas', 'S'),
 ('American Samoa', 'O'),
 ('Arizona', 'W'),
 ('California', 'W'),
 ('Colorado', 'W'),
 ('Connecticut', 'N'),
 ('District of Columbia', 'N'),
 ('Delaware', 'N'),
 ('Florida', 'S'),
 ('Georgia', 'S'),
 ('Guam', 'O'),
 ('Hawaii', 'O'),
 ('Iowa', 'M'),
 ('Idaho', 'W'),
 ('Illinois', 'M'),
 ('Indiana', 'M'),
 ('Kansas', 'M'),
 ('Kentucky', 'S'),
 ('Louisiana', 'S'),
 ('Massachusetts', 'N'),
 ('Maryland', 'N'),
 ('Maine', 'N'),
 ('Michigan', 'W'),
 ('Minnesota', 'M'),
 ('Missouri', 'M'),
 ('Northern Mariana Islands', 'O'),
 ('Mississippi', 'S'),
 ('Montana', 'W'),
 ('National', 'O'),
 ('North Carolina', 'S'),
 ('North Dakota', 'M'),
 ('Nebraska', 'W'),
 ('New Hampshire', 'N'),
 ('New Jersey', 'N'),
 ('New Mexico', 'W'),
 ('Nevada', 'W'),
 ('New York', 'N'),
 ('Ohio', 'M'),
 ('Oklahoma', 'S'),
 ('Oregon', 'W'),
 ('Pennsylvania', 'N'),
 ('Puerto Rico', 'O'),
 ('Rhode Island', 'N'),
 ('South Carolina', 'S'),
 ('South Dakota', 'M'),
 ('Te

In [47]:
regiondict = dict(region_list)

In [58]:
regioncounty['region'] = regioncounty['State'].map(regiondict)

In [49]:
regioncounty['State'] = regioncounty['State'].str.replace('-',' ')

In [64]:
subset2 = regioncounty[['State', 'County', 'Avg_Income', 'region']]
tuples2 = [tuple(x) for x in subset2.values]

In [65]:
def insert_datas(dataframe):
    tuples = list(dataframe.itertuples(index=False, name=None))
    insert_statement = """INSERT INTO countyincomes2 (State,
                                               County,
                                               Avg_Income,
                                               region)
                           VALUES (%s, %s, %s, %s);"""
    cursor.executemany(insert_statement, tuples)
    cnx.commit()

In [66]:
insert_datas(regioncounty)

In [59]:
regioncounty

,State,County,Avg_Income,region
0,Alabama,Autauga County,53682.0,S
1,Alabama,Baldwin County,50221.0,S
2,Alabama,Barbour County,32911.0,S
3,Alabama,Bibb County,36447.0,S
4,Alabama,Blount County,44145.0,S
5,Alabama,Bullock County,32033.0,S
6,Alabama,Butler County,29918.0,S
7,Alabama,Calhoun County,39962.0,S
8,Alabama,Chambers County,32402.0,S
9,Alabama,Cherokee County,34907.0,S


In [60]:
null_columns = regioncounty.columns[regioncounty.isnull().any()]

In [61]:
pd.DataFrame(null_columns)

,0


In [62]:
regioncounty[regioncounty['region'].isnull()].index.tolist()

[]

In [63]:
regioncounty.iloc[1819]

State                New Mexico
County        San Miguel County
Avg_Income                28275
region                        W
Name: 1819, dtype: object

### Use Latitude & Longitude to find County

In [82]:
pip install geopy

     |████████████████████████████████| 102kB 2.5MB/s ta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
from geopy import geocoders
from geopy.geocoders import Nominatim

In [2]:
import config

In [3]:
geolocator = Nominatim(user_agent=config.host)
location = geolocator.reverse("41.0391, -107.653")
print(location.address)

Baggs, Carbon County, Wyoming, 82321, United States


In [103]:
location.address

'Execution Rocks Light, Sands Light Road, Sands Point, Nassau County, New York, 11050, United States'

In [106]:
list_address = location.address.split(',')

In [109]:
county = list_address[3].strip()

In [110]:
county

'Nassau County'

In [119]:
AZ.address

'Main Street, Ferndale, Humboldt County, California, United States'

In [6]:
AZ = geolocator.reverse("42.449274, -90.055652")
AZ.address

'Apple River Canyon State Park, River Route Nature Trail, Millville, Jo Daviess County, Illinois, United States'